In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import subSample, getDataFromJSON, trainer, getTotalAccuracy

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Change this to cuda for GPU enabled computers

ImportError: cannot import name 'getTotalAccuracy' from 'normalizer' (/home/robjohnson/School/research/DeepLearningSolarFlares/normalizer.py)

## Load the data ##

In [ ]:
# This file has 77270 data points. 71633 do not have NAN values.
# %time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=-1)
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

In [ ]:
# This file has 93767 data points. 82425 of them do not have NAN values.  
# %time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=-1)
train2, labels2 = subSample("data/train_partition2_data.json", device=device)

In [ ]:
# This file has 42986 data points. 37759 of them do not have NAN values.  
# %time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=-1)
train3, labels3 = subSample("data/train_partition3_data.json", device=device)

In [ ]:
train2[0,1]

In [ ]:
train2[0,0]

## Define the network ##

In [ ]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
# from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.layerout(y)
        y = self.smax(y)
        return y



## Adjust hyper parameters with 3 fold validation ##

In [ ]:
modelArgs = [4096*4, 2048*4]
modelKwargs = {'drop1':.25}
trainKwargs = {'epochs':50, 'lr':0.0001}

In [ ]:
train1Args = [logRegWithHidden, torch.cat((train1, train2), dim=0), labels1 + labels2, None, train3, labels3, None]
%time train12 = trainer(*train1Args, *modelArgs, **trainKwargs, **modelKwargs)

In [ ]:
train13 = trainer(
    logRegWithHidden,
    torch.cat((train1, train3), dim=0),
    labels1 + labels3,
    None,
    train2,
    labels2,
    None,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

In [ ]:
train23 = trainer(
    logRegWithHidden,
    torch.cat((train3, train2), dim=0),
    labels3 + labels2,
    None,
    train1,
    labels1,
    None,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

In [ ]:
acc = getTotalAccuracy(train12, train23, train13, 
                       torch.cat((train1, train2, train3), dim=0)
                      labels1 + labels2 + labels3)
print('Average accuracy of all three models on all data: ', acc)

## Final model creation for testing ##

Once acceptable hyperparameters have been established, run this code to train on all the data and print out a CSV that predicts from the test data.

In [ ]:
train123 = trainer(
    logRegWithHidden,
    torch.cat((train1, train2, train3), dim=0),
    labels1 + labels2 + labels3,
    None,
    torch.cat((train1, train2, train3), dim=0),
    labels1 + labels2 + labels3,
    None,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

In [ ]:
from datetime import datetime
def tester(model, pathToWrite=None):
    if pathToWrite is None:
        pathToWrite = f'results/submission{datetime.now().strftime("%d_%H:%M")}.csv'
    # Get test data
    test, ids, _ = getDataFromJSON(path='data/test_4_5_data.json', test=True, device=device)
    # get our guesses from the network
    guesses = torch.argmax(model(test))
    assert len(ids) == guesses.shape
    # Open a file to write to
    file = open(pathToWrite, mode='w')
    print('Id,Label', file=file)
    for i in range(len(ids)):
        print(ids[i], guesses[i], sep=',', file=file)
    file.close()

tester(train123)

In [ ]:
from datetime import datetime
acc = 
PATH = f'savedModels/lr{lr}acc{acc}time{datetime.now().strftime("%d_%H:%M")}.pth'
torch.save(newModel.state_dict(), PATH)
print('REMEMBER TO DELETE YOUR ACCURACY SO THE NEXT PERSON REMEMBERS TO WRITE THEIRS')